# GWAS Preparasi

In [1]:
pwd()

'/home/hnasrulloh/Works/pangenomes/notebooks'

In [2]:
from os import path

import pandas as pd


In [3]:
DATA_DIR = path.join(path.pardir, 'data')

SAMPLES_DIR = path.join(DATA_DIR, 'samples')
PAN_DIR = path.join(DATA_DIR, 'pangenomes')

GFF_DIR = path.join(DATA_DIR, 'samples_combined')

RIFAMPIN_GFF_DIR = path.join(DATA_DIR, 'gff_rifampin')
ISONIAZID_GFF_DIR = path.join(DATA_DIR, 'gff_isoniazid')
ETHAMBUTOL_GFF_DIR = path.join(DATA_DIR, 'gff_ethambutol')

## 1. Trait/Phenotype tsv files

In [4]:
AMR_EXCEL = path.join(DATA_DIR, 'amr_ready.xlsx')
df = pd.read_excel(AMR_EXCEL, dtype=str)

df = df.loc[:, ['genome_id', 'antibiotic', 'phenotype']]

pheno = { 'R': 1, 'S': 0 }
df['phenotype'] = df['phenotype'].map(pheno)

# Match the samples name within gene_presence_absence.Rtab
samples = df.genome_id.map(lambda name: name + '.PATRIC')
df.insert(0, 'samples', samples)
df = df.drop(['genome_id'], axis=1)

for antibiotic in ['ethambutol', 'rifampin', 'isoniazid']:
    df_ab = df[df.antibiotic == antibiotic]
    df_ab = df_ab.drop(['antibiotic'], axis=1)
    df_ab.columns = ['samples', 'resistance']

    ANTIBIOTIC_TSV = path.join(DATA_DIR, f'traits_{antibiotic}.tsv')
    df_ab.to_csv(ANTIBIOTIC_TSV, index=False, sep='\t')

## 2. Pisahkan GFF file berdasarkan antibiotic

In [5]:
def used_gff_filenames(antibiotic):
    GENE_PRESENCES = path.join(PAN_DIR, antibiotic, 'gene_presence_absence.csv')
    gffs = pd.read_csv(GENE_PRESENCES, low_memory=False).columns[3:]
    return list(map(lambda name: f'{name}.gff', gffs))

In [6]:
for antibiotic in ['rifampin', 'isoniazid', 'ethambutol']:
    gffs = used_gff_filenames(antibiotic)
    
    with open(path.join(DATA_DIR, f'gff_{antibiotic}_panfeed1.txt'), 'w') as f:
        f.write('\n'.join(gffs))

Run `panfeed`! Lihat `cmd.md`